# Upload Data to MyCoast


(https://gist.github.com/pugwonk/ec192de6814486db678bdbb8fed0937b)

## 1. Start with Airtable

In [31]:
import pandas as pd

survey_link = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Survey_wPhotoLinks.csv"
photos_folder = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized"

df = pd.read_csv(survey_link)
df = df.drop(['Photo','S123_Object_ID'], axis=1 )
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN


Combine tabular data with file names for photos. 

In [32]:
import os
all_photos = os.listdir( photos_folder )

for i,r in df.iterrows():
    row_id = r['unique_id']
    
    rows_photos = [ i for i in all_photos if i.split('-')[0] == str(row_id) ]

    df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)

df.head()

C:\Users\cansu\AppData\Local\Temp\ipykernel_23396\1457632312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2-0.jpg, 2-1.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)


,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,"2-0.jpg, 2-1.jpg"
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,3-0.jpg
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,4-0.jpg
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,5-0.jpg
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,6-0.jpg


# Resize images  
Make sure **NOT** to work on the original images, create a new resize_folder.  
This code overrides the exsisting files in the folder. 

`max_size` defines the max size allowed on either dimension of the image.

In [33]:
from PIL import Image
max_size = 720

# There are reports without images attached to them.
for i,r in df[df['Photo_filenames']!='' ].iterrows():

    row_id = r['unique_id']
    row_photos = r['Photo_filenames'].split(', ')

    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]

    for img_path in row_photos_fullpath: 
        if img_path != '': 
            image = Image.open( img_path )

            # Image dimensions 
            x = image.size[0]
            y = image.size[1]
            ratio = x/y

            if (x > max_size) | (y > max_size ):
                # Make sure the image is RGB. 
                image = image.convert('RGB')

                # limit longest side to 1000px
                if x > y:
                    new_x = max_size
                    new_y = int(max_size/ratio)
                else:
                    new_y = max_size
                    new_x = int(max_size*ratio)

                try:
                    image_resized = image.resize( (new_x, new_y) )
                    image_resized.save( img_path )
                except:
                    print( "Error resizing image: ", img_path )
                    image.save( img_path )



Clean up Source column. It is not written well.
**Overland flooding (Nearby waterbody)**  

Overland Flooding  
Overland Flooding Water Coming  

**Rainfall (Water pooling from direct rainfall)**  
Heavy Rains And No Known Source Of Drainage  
Heavy Rain And Storm Drains  
Rainfall Water Pooling  

**Storm drain (water pooling near a storm drain)**  
Storm Drainage Source  
Storm Drain/ Catch Basin  
Storm Drain Water  

**Broken pipes/water main break**  
Flooding From Pipes  

**Unsure**  
Not Sure  
Multiple Causes  

**Other**  


In [34]:
df['Source'] = df['Source'].str.replace('0','' )
df['Source'] = df['Source'].str.replace('_',' ' )
df['Source'] = df['Source'].str.replace('other','Other' )
df['Source'] = df['Source'].str.strip()
df['Source'] = df['Source'].fillna( '' )

df['Source'] = df['Source'].str.replace( 'Overland Flooding' , 'Overland flooding (Nearby waterbody)' )
df['Source'] = df['Source'].str.replace( 'overland flooding flooding from' , 'Overland flooding (Nearby waterbody)' )
df['Source'] = df['Source'].str.replace( 'overland flooding water coming' , 'Overland flooding (Nearby waterbody)' )

df['Source'] = df['Source'].str.replace( 'Heavy Rains and No known source of drainage' , 'Rainfall (Water pooling from direct rainfall)' )
df['Source'] = df['Source'].str.replace( 'Heavy Rain and Storm Drains' , 'Rainfall (Water pooling from direct rainfall)' )
df['Source'] = df['Source'].str.replace( 'rainfall water pooling from dir' , 'Rainfall (Water pooling from direct rainfall)' )

df['Source'] = df['Source'].str.replace( 'storm drain water coming from s' , 'Storm drain (water pooling near a storm drain)' )
df['Source'] = df['Source'].str.replace( 'Storm Drainage Source' , 'Storm drain (water pooling near a storm drain)' )
df['Source'] = df['Source'].str.replace( 'Storm Drain/ Catch Basin' , 'Storm drain (water pooling near a storm drain)' )

df['Source'] = df['Source'].str.replace( 'Flooding from pipes' , 'Broken pipes/water main break' )

df['Source'] = df['Source'].str.replace( 'multiple causes if multiple cau' , 'Unsure' )
df['Source'] = df['Source'].str.replace( 'not sure' , 'Unsure' )

[i for i in df['Source'].unique()]

['Overland flooding (Nearby waterbody)',
 'Storm drain (water pooling near a storm drain)',
 'Rainfall (Water pooling from direct rainfall)',
 '',
 'Unsure',
 'Broken pipes/water main break',
 'Other']

In [35]:
df['Approx. Depth'] = df['Approx. Depth'].str.replace('in','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace('_ches','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace(' ches','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace('_','-') 

df['Approx. Depth'].unique()

array([nan, '6', '4', '8', '5', '2', '15', '12', '16', '3', '10', '8-12',
       '20', '0', '9', '18', '4.75', '14.5', '7', '6.5', '3.5', '1',
       '0.5', '2 ', '1-6', '6-12', '12-24', '<1', '24', '17', '11', '19'],
      dtype=object)

In [36]:
import os
import uuid
import requests
import json
import time

def isnan(value):
  try:
      import math
      return math.isnan(float(value))
  except:
      return False

myCoastUserId = 13213
myCoastUserHash  = "$P$BEnD9bUs3T4TcaMykJ7D/TnAjVmekp1"

# ---------------- Open and Upload Photos --------------
for i,r in df[ df['Photo_filenames'].notnull() ].sample(12).iterrows():

    row_id = r['unique_id']

    row_photos = r['Photo_filenames'].split(', ')
    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]
    image_ids = [str(uuid.uuid4()) for _ in row_photos_fullpath ] # generate unique ids for each image

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*" }

    # Define the URL
    for image_id, full_path in zip(image_ids , row_photos_fullpath):

        url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=image&id={image_id}&purpose=0"
        
        with open (full_path, 'rb') as file:
            file_content = file.read()
            response = requests.post( url, files = {'upfile': ('upfile', file_content)} , headers=headers )

            # Print the response
            if response.status_code == 200:
                print( "OK" )
            else:
                print(f"Photo: Request failed with status code {response.status_code}")
                print( response.text )

    # ---------------------- REPORT ----------------------
    # Define the URL
    report_url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=report"

    myReport = [
        {"name":"_gform-form-id", "value": 72 },
        {"name":"from_device", "value": "Can reports" },
        {"name":"photo_date", "value": r['Date'].strftime("%Y-%m-%d") },
        {"name":"photo_time", "value": r['Date'].strftime("%I:%M %p").lower() },
        {"name":"alternate-location-name", "value": r['Neighborhood'] },
        {"name":"image1", "value": image_ids },
        {"name":"guessCause", "value": r['Source'] },
        {"name":"cause-other", "value": r['What are additional causes of flooding?'] },
        {"name":"flooded-other", "value": r["What is affected by the flood waters?"] },
        {"name":"guessDepth", "value": r['Approx. Depth'] },
        {"name":"post_comment", "value": r['Additional Comments'] },
        {"name":"location_longitude", "value": float(r['Longitude']) },
        {"name":"location_latitude", "value": float(r['Latitude']) },
    ]

    for report_item in myReport:
        if (report_item['name'] == 'cause-other') or (report_item['name'] == 'flooded-other') :
            if isnan( report_item['value']):
                report_item['value'] = []
        else:
            if isnan( report_item['value']):
                report_item['value'] = ""

    [ print(f"{i['name']} = {i['value']}") for i in myReport ]
    print( " ------------------- " )
    myReport = json.dumps(myReport)
    
    # Generate a unique report ID
    report_id = str(uuid.uuid4())

    # Send the POST request with JSON payload
    report_url = f"{report_url}&id={report_id}"

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*", "Content-Type": "application/json" }
    response = requests.post( report_url , data=myReport , headers=headers )

    # Print the response
    if response.status_code == 200:
        print("Report OK")
    else:
        print(f"Report: Request failed with status code {response.status_code}")

    time.sleep(5)


OK
_gform-form-id = 72
from_device = Can reports
photo_date = 2019-09-27
photo_time = 09:30 am
alternate-location-name = Howard Beach/ Hamilton Beach
image1 = ['fedaae5f-6b9e-4fdf-a8df-34c266e24c7d']
guessCause = Overland flooding (Nearby waterbody)
cause-other = []
flooded-other = []
guessDepth = 0.5
post_comment = 
location_longitude = -73.8302304
location_latitude = 40.6513987
 ------------------- 
Report OK
OK
OK
_gform-form-id = 72
from_device = Can reports
photo_date = 2019-10-27
photo_time = 07:45 am
alternate-location-name = Howard Beach/ Hamilton Beach
image1 = ['b82d4ba1-cb06-4639-a343-b3ca404040bc', 'bef44d83-138a-4319-af85-fa038c437283']
guessCause = 
cause-other = []
flooded-other = []
guessDepth = 
post_comment = 
location_longitude = -73.8317401
location_latitude = 40.6499525
 ------------------- 
Report OK
OK
_gform-form-id = 72
from_device = Can reports
photo_date = 2018-11-26
photo_time = 08:00 am
alternate-location-name = Howard Beach/ Hamilton Beach
image1 = ['1c7f8

# Survey123

In [84]:

df_survey = pd.read_csv( r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\Survey_Data.csv" )
survey123_photos = pd.read_csv( r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\GlobalID_Compare.csv" ) 

display(survey123_photos.head())

df_survey = df_survey.drop( ['Creator','EditDate','Editor'] , axis=1 )
df_survey = df_survey.dropna( axis=0 , subset=['globalid'] )

df_survey.head()

,ATTACHMENTID,GLOBALID,REL_GLOBALID,CONTENT_TYPE,ATT_NAME
0,20,{1EAC122B-989F-468D-8253-366753782318},{E4EC81F4-186D-41B0-8779-A0E938C814A3},image/jpeg,2.jpeg
1,21,{813D68D7-3524-4597-8127-4D59B75243B3},{934FAB2F-3314-4ABC-8583-FF1A3B968439},image/jpeg,3.jpeg
2,22,{98938CA5-BA06-4A1C-8007-42657A5B17A0},{0A4DB08D-AE85-441E-86B2-80842C85C4C9},image/jpeg,20200506_202850.jpg
3,23,{7ECA63C5-F6E3-4A6E-85D8-93BBDAF858F3},{79733B16-EDA5-4D37-831A-816FBF3AB2E3},image/jpeg,15888119669837837470918754484373.jpg
4,24,{41950B37-E65B-4897-82D8-5EEA879A3966},{EFC6E186-66BE-4E7C-866D-0F7EEC330779},image/jpeg,Screenshot_20200506-204102_Bimostitch.jpg


,objectid,globalid,CreationDate,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,x,y
0,20,{D67D428A-38B6-4411-90E4-BCC40B6FF679},2020/05/07 00:12:32,property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.837982,40.657893
1,21,{1C95B9A1-FCFA-41F5-B9F5-150063806F39},2020/05/07 00:15:23,roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.837842,40.657755
2,22,{1143F1AD-279B-4126-AD59-1B57698DEFD9},2020/05/07 00:35:38,"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020/05/07 00:00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,-73.829444,40.653456
3,23,{8687F8A1-B09C-482C-9BFB-60E44565AA0F},2020/05/07 00:41:22,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,6th road facing crossbay blvd,"P. Meyers, Meyers Emergency Management",NaN,00:39,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.817104,40.609023
4,24,{A92174F2-8A4D-4F37-8B44-4EDF35AD676D},2020/05/07 00:46:56,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.829534,40.653500


In [153]:
# use fiona to open GDB file

path = r"C:\Users\cansu\Downloads\Survey123_Floodwatch_724entries(1)\Survey123_Floodwatch_724entries\S123_b9b1d621d16543378b6d3a6b3e02b424_FGDB\520636cb-9aef-45eb-9524-4bf049dd35fa.gdb"

import geopandas as gpd
import fiona
layers = fiona.listlayers( path )
print( layers )

gdf_survey = gpd.read_file( path , driver='FileGDB' , layer = 'survey')
photos = pd.read_csv(r"C:\Users\cansu\Downloads\deneme\pho1.csv")

photos_list = gdf_survey[['globalid']].join( photos.set_index('REL_GLOBALID') , on='globalid', how='right' )
photos_list['GLOBALID'] = photos_list['GLOBALID'].apply( lambda x: x.replace("{","").replace("}","") )
gdf_survey = gdf_survey.set_index('globalid').join( photos_list.groupby('globalid').agg({'GLOBALID':list}) )
gdf_survey = gdf_survey.drop(['Creator','EditDate','Editor'], axis=1 )
gdf_survey = gdf_survey.to_crs( 4326 )
gdf_survey.head(3)

['survey', 'survey__ATTACH']


,CreationDate,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry,GLOBALID
globalid,,,,,,,,,,,,,,,,,
{E4EC81F4-186D-41B0-8779-A0E938C814A3},2020-05-07 00:12:32+00:00,property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83798 40.65789),[1EAC122B-989F-468D-8253-366753782318]
{934FAB2F-3314-4ABC-8583-FF1A3B968439},2020-05-07 00:15:23+00:00,roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83784 40.65775),[813D68D7-3524-4597-8127-4D59B75243B3]
{0A4DB08D-AE85-441E-86B2-80842C85C4C9},2020-05-07 00:35:38+00:00,"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020-05-07 00:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,POINT (-73.82944 40.65346),[98938CA5-BA06-4A1C-8007-42657A5B17A0]


In [214]:
ntacd = gpd.read_file( r"D:\WORK\FLOODNET\Local_Reports\Data\NYCDTA_Boundaries_4326_Longdata.geojson" )
ntacd = ntacd[['labels','geometry']].copy()
ntacd['labels'] = ntacd['labels'].str.strip()

# gdf_survey_Carrol = gdf_survey[ gdf_survey.intersects( ntacd[ ntacd['labels'].str.contains('Carrol') ].to_crs(2263).buffer( 5280*1 ).to_crs(4326).values[0] ) ]
gdf_survey_Carrol = gdf_survey.copy()

gdf_survey_Carrol['CreationDate'] = pd.to_datetime( gdf_survey_Carrol['CreationDate'] ).dt.tz_localize(None)


# one_photo = gdf_survey_Carrol[ ( gdf_survey_Carrol['CreationDate'] > pd.to_datetime('2021-08-30') ) & ( gdf_survey_Carrol['CreationDate'] < pd.to_datetime('2021-09-15') ) ]
one_photo = gdf_survey_Carrol[ (gdf_survey_Carrol['please_indicate_the_approximate'] > 11) & (gdf_survey_Carrol['please_indicate_the_approximate'] < 13) ]

# one_photo = gdf_survey[ gdf_survey.intersects( ntacd[ ntacd['labels'].str.contains('Carrol') ].to_crs(2263).buffer( 5280*3 ).to_crs(4326).values[0] ) ]
t = one_photo[one_photo['please_indicate_the_approximate'].notnull() ].sort_values('CreationDate')

pd.set_option('display.max_colwidth', None)
display(t[['additional_comments','GLOBALID']])




,additional_comments,GLOBALID
globalid,,
{7D511030-C159-4DC6-8EA5-6E82E2AD63E9},NaN,"[62202680-A75D-4680-89DC-248063B5A089, 28513DC4-9B2F-4167-8FB7-078C06A1D9BA]"
{B12F5BC2-85FB-4458-8418-BD811D5C7CD9},NaN,"[A9450080-2CDA-4131-86A5-CE26E31A12B2, E03DF52B-FD0D-47FC-8520-E01036471DBC]"
{C0D84E67-E11E-4BFF-8D4F-6CC74DA409D2},NaN,"[032FDC49-9DC9-41F0-84DD-09A49EF9CFDE, 77BF94C1-E764-48AC-8214-573E9080EEE8]"
{8E36ADC6-94D2-45B1-8581-423EC52DA2A8},NaN,[FDE2827A-8886-4160-87EE-FA45A27BA4FA]
{EFAA2F28-DCA0-45D8-8E35-CA9AD08758E4},NaN,[5FAC95E2-14C8-4F37-83FE-A171F5F0FA25]
{43214696-A9C8-4DEE-8E1F-2D497A3C513B},"Up to 2nd Step, Has good progression photos\n","[3C4D6210-FFC2-4D73-84F4-3ACE9C64278E, D4B187D8-ABF1-4DC6-81C4-3C3D7434BD7C, 8E489537-5057-4554-8D72-196699B36949]"
{C6C0A1A9-4C7C-4FBA-8262-2C51FF3EC222},NaN,[B4F631D3-AD53-49CD-8800-10E34C49F512]
{B7A2FE4B-A4C6-430B-80D3-780720425928},NaN,[87139D6F-5709-4BE0-8B90-52178DFD0602]
{1F50E479-1C9F-4DDE-8C44-6FFFC9E45452},NaN,[FBE0799C-E828-49B4-8EDB-B4FA76CFD3E0]


In [159]:
ntacd = gpd.read_file( r"D:\WORK\FLOODNET\Local_Reports\Data\NYCDTA_Boundaries_4326_Longdata.geojson" )
ntacd = ntacd[['labels','geometry']].copy()

ntacd.head()

gdf_survey_N = gpd.sjoin( gdf_survey , ntacd , how='left' , predicate='within' )
gdf_survey_N = gdf_survey_N.dropna( axis=0, subset=['labels'] )

gdf_survey_N['CreationDate'] = pd.to_datetime( gdf_survey_N['CreationDate'] ).dt.tz_localize(None)

gdf_survey_N[ gdf_survey_N['labels'].str.contains('Edge') ]

,CreationDate,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry,GLOBALID,index_right,labels
globalid,,,,,,,,,,,,,,,,,,,
{16AE590D-126D-4319-87C1-3986F90BBE59},2020-09-23 20:15:39,roads_streets,NaN,"sunny,light_wind",NaN,NaN,jasimov@riserockaway.org,3.0,15:00,2020-09-19 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.78890 40.59215),"[FAB10188-DDB3-45BC-8B9C-BB19873313E2, 863F055...",19.0,Rockaway Beach-Arverne-Edgemere
{11D31E5B-5102-426A-8A79-C51567A411F9},2020-09-24 15:16:30,"roads_streets,parking_lots",NaN,"sunny,light_wind",NaN,NaN,KG,3.0,17:70,2020-09-23 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.78814 40.59214),[EFBF2484-CB1E-4A86-835B-F04FC78BDB85],19.0,Rockaway Beach-Arverne-Edgemere
{5909CA62-C7EF-4D6F-8872-0A9190E9FD21},2020-10-12 23:24:42,"roads_streets,sidewalks",NaN,"light_rain,heavy_rain",NaN,this particular area has been flooding for yea...,Brody Ajemian (917) 620-2636,5.0,21:30,2020-10-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.81421 40.58756),"[AA6FD33A-498A-4891-854F-6EE324FCAC59, F51192B...",19.0,Rockaway Beach-Arverne-Edgemere
{5BD02B00-C83F-4B49-8AF3-4A12A77AA7EB},2021-01-20 20:17:08,"roads_streets,property",NaN,"sunny,light_wind",NaN,NaN,kg,4.0,14:02,2021-01-16 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.81067 40.58963),"[70544E9B-0CF3-47B7-8681-AC74B6F3BFF9, F6B3900...",19.0,Rockaway Beach-Arverne-Edgemere
{6C9010B5-8EED-48DA-893F-B636AC669FB7},2021-01-20 20:19:06,roads_streets,NaN,"sunny,light_wind",NaN,NaN,kg,2.0,14:04,2021-01-16 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.81110 40.58954),[07AED4A5-E18F-4494-8CC3-02358F270D24],19.0,Rockaway Beach-Arverne-Edgemere
{A13F118B-6480-4B77-81B4-F268E523AAE0},2021-02-02 23:34:34,"roads_streets,property",NaN,heavy_wind_gusts,NaN,NaN,NaN,6.0,11:30,2021-02-02 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.81242 40.59030),[195BCA1B-DCE0-4641-838E-C60CDD687584],19.0,Rockaway Beach-Arverne-Edgemere
{027B0AA6-FD50-49FD-8835-A10EE1706CC8},2021-02-02 23:37:33,"roads_streets,property",NaN,heavy_wind_gusts,NaN,NaN,NaN,8.0,23:58,2021-02-01 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.81246 40.59026),[DEE3008F-E60C-421E-88CF-1083731F241B],19.0,Rockaway Beach-Arverne-Edgemere
{A8D25B45-09C1-46E3-8378-862411532357},2021-07-23 00:18:10,"roads_streets,sidewalks,property",NaN,light_wind,NaN,NaN,Nick Culver,4.0,19:50,2021-07-22 16:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,Yes,NaN,POINT (-73.81018 40.59080),[0743FD4A-3B28-485A-899F-BEAB04695CAD],19.0,Rockaway Beach-Arverne-Edgemere
{292DE4E1-77F5-4D9D-8D0D-586730431655},2021-08-22 17:32:32,"roads_streets,sidewalks,parking_lots",NaN,light_rain,NaN,NaN,K. Lawrence,NaN,08:17,2019-11-13 16:00:00+00:00,"Overland_flooding_Water_coming_,Rainfall_Water...",NaN,Yes,Yes,NaN,POINT (-73.78836 40.59212),"[1A4CB48E-A75F-4D1A-885A-EF6AF473F6B6, 5B9DAF4...",19.0,Rockaway Beach-Arverne-Edgemere
